### Folium

In [0]:
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Import Folium
import folium

# Create a map centered around the average latitude and longitude
map_center = [pandas_df['latitude'].mean(), pandas_df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Add points to the map
for _, row in pandas_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Name: {row['name']}, Population: {row['population']}, State: {row['state']}"
    ).add_to(m)

# Display the map
m